# Pystencils Multi Phase LBM Kernel - Performance and Volumes

In [ ]:

import sys 
sys.path.append('../pystencils')
sys.path.append('../../pystencils')
sys.path.append('../warpspeed')
sys.path.append('../measutils')

%load_ext autoreload
%autoreload 1

%aimport plot_utils
%aimport multi_phase_lbm
%aimport pystencilswarpspeedkernel

In [ ]:
import math
import time

from pystencils.session import *

import multi_phase_lbm

from pystencilswarpspeedkernel import PyStencilsWarpSpeedKernel, getFieldExprs, lambdifyExprs, simplifyExprs

from griditeration import *
from predict_metrics import *
from volumes_isl import *
from plot_utils import *


import random

import pycuda.autoinit
import pycuda.driver as drv
from meas_utils import benchKernel
from plot_utils import *
from meas_utils import *
from measured_metrics import MeasuredMetrics, ResultComparer

from meas_db import MeasDB

meas_db = MeasDB("multiphaselbm.db")
#meas_db.clearDB()

In [ ]:

ast = multi_phase_lbm.get_allen_cahn_ast((64,4,1))

wsKernel = PyStencilsWarpSpeedKernel(ast)
print()
def printFields(fields):
    for field in fields:
        print(field)
        for exp in fields[field]:
            print(exp)


printFields(wsKernel.getLoadExprs3D())
printFields(wsKernel.getStoreExprs3D())
printFields(wsKernel.genLoadExprs())

In [ ]:
predValues = dict()
measValues = dict()

device = selectDevice(drv.Device(0).name())
print(device.name)

def nextBlockSize():
    for xblock in [1, 2, 4, 8, 16, 32, 64, 128, 256, 512, 1024]:
        for yblock in [1, 2, 4, 8, 16, 32, 64, 128, 256, 512, 1024]:
            for zblock in [1, 2, 4, 8, 16, 32, 64]:
                if xblock*yblock*zblock not in [256, 512]:
                    continue
                yield (xblock, yblock, zblock)    
                

for block in nextBlockSize():

    for r in [1]:
        key = (r, *block)
        lc, basic, meas = meas_db.getEntry(r, block, (1,1,1), multi_phase_lbm.domain_size, device)

        #basic = None
        #meas = None
        
        if meas is None or basic is None:        
            print("measured")

            if r == 0:
                ast = multi_phase_lbm.get_hydro_lb_ast(block[:3])
                update_rule = multi_phase_lbm.hydro_lb_update_rule 
            elif r == 1:
                ast = multi_phase_lbm.get_allen_cahn_ast(block[:3])
                update_rule = multi_phase_lbm.allen_cahn_lb 
            elif r == 2:
                update_rule = None
                ast = multi_phase_lbm.get_lbm_ast(block[:3])                    

                
            kernel = ast.compile()
            wsKernel = PyStencilsWarpSpeedKernel(ast)
            wsKernel.registers = kernel.num_regs
            runFunc = partial(multi_phase_lbm.dh.run_kernel, kernel)

            lc = LaunchConfig.compute(wsKernel, block, multi_phase_lbm.domain_size, (1,1,1), device)
            
            if meas is None:
                meas = MeasuredMetrics.measure(runFunc, lc)

            if basic is None:
                basic = BasicMetrics.compute(lc, device, wsKernel)
            

            #meas_db.insertValue(r, block, (1,1,1), device, basic, meas, lc)
        else:
            print("cached")

        metrics = DerivedMetrics(lc, basic, device, meas)

        measValues[key] = meas
        predValues[key] = metrics

        print(str(lc), end="")
        print(str(basic), end="--\n")
        rc = ResultComparer(meas, metrics)
        print(str(rc))              

        print()
        #meas_db.commit()


In [ ]:
volumeScatterPlot([(k[1:], measValues[k].memLoad, predValues[k].memLoadV1, k[0]) for k in measValues], "Multi Phase LBM Memory Load Volumes V1")
volumeScatterPlot([(k[1:], measValues[k].memLoad, predValues[k].memLoadV2, k[0], predValues[k].memLoadV1) for k in measValues], "Multi Phase LBM Memory Load Volumes V2")
volumeScatterPlot([(k[1:], measValues[k].memLoad, predValues[k].memLoadV3, k[0], predValues[k].memLoadV2) for k in measValues], "Multi Phase LBM Memory Load Volumes V3")
volumeScatterPlot([(k[1:], measValues[k].memLoad, predValues[k].memLoadV4, k[0], predValues[k].memLoadV3) for k in measValues], "Multi Phase LBM Memory Load Volumes V4")

In [ ]:

volumeScatterPlot([(k[1:], measValues[k].L2Load_tex, predValues[k].L2LoadV1, k[0]) for k in measValues], "Multi Phase LBM L2 Load Volumes V1")
volumeScatterPlot([(k[1:], measValues[k].L2Load_tex, predValues[k].L2LoadV2, k[0], predValues[k].L2LoadV1) for k in measValues], "Multi Phase LBM L2 Load Volumes V2")

In [ ]:
volumeScatterPlot([(k[1:], measValues[k].L2Store, predValues[k].L2Store, k[0]) for k in measValues], "Multi Phase LBM L2 Store Volumes V1")
volumeScatterPlot([(k[1:], measValues[k].memStore, predValues[k].memStoreV1, k[0]) for k in measValues], "Multi Phase LBM Memory Store Volumes V1")
volumeScatterPlot([(k[1:], measValues[k].memStore, predValues[k].memStoreV2, k[0], predValues[k].memStoreV1, ) for k in measValues], "Multi Phase LBM Memory Store Volumes V2")

In [ ]:
scatterPlot([(k[1:4], min(20000, predValues[k].TLBpages) , measValues[k].lups * (measValues[k].memLoad + measValues[k].memStore)) for k in measValues.keys() if predValues[k].limPheno > 0 ], "testplot")
scatterPlot([(k[1:4], 1-1/(k[0]*k[1]), measValues[k].lups * (measValues[k].memLoad + measValues[k].memStore)) for k in measValues.keys() if predValues[k].limPheno > 0 ], "testplot")

In [ ]:
volumeScatterPlot([(k[1:4], measValues[k].L1Wavefronts*32, predValues[k].L1Cycles) for k in measValues], "L1 Cache Cycles")

In [ ]:
fig, ax = plt.subplots()
fig.set_figwidth(4)
fig.set_figheight(4)
fig.set_dpi(150)

for r in [0, 1]:
    keys = [k for k in measValues]
    ax.plot([predValues[k].smL1Alloc  /  (128*1024) for k in keys],
            [(measValues[k].L2Load_tex - predValues[k].L2LoadV1) / (predValues[k].L1Load - predValues[k].L2LoadV1) for k in keys], ".", alpha=0.2)
    

values = np.arange(0.0, 10.0, 0.1)


#ax.plot (values, 0.45*np.exp(-9.0*np.exp(-0.8*values))) 
#ax.plot (values, 0.43*np.exp(-9.0*np.exp(-0.65*values))) 
#ax.plot (values, 0.25*np.exp(-9.0*np.exp(-0.5*values))) 

ax.axvline(1.0)

In [ ]:
fig, ax = plt.subplots()
fig.set_figwidth(4)
fig.set_figheight(4)
fig.set_dpi(150)

for r in [1,2,4,8,16,32,64,128,256,512]:
    keys = [k for k in measValues if k[1] == r]
    ax.plot([predValues[k].waveL2Alloc / 6 /1024 / 1024 for k in keys],
            [(measValues[k].memLoad - predValues[k].memLoadV1) / predValues[k].L2LoadV2 for k in keys], ".", alpha=0.2)
    

values = np.array(range(1024, 1024*1024, 1024))


#ax.plot (values, 0.43*np.exp(-9.0*np.exp(-0.0000065*values))) 
#ax.plot (values, 0.25*np.exp(-9.0*np.exp(-0.000005*values))) 

ax.axvline(1)

In [ ]:
for key in predValues.keys():
    break
    r, xblock, yblock, zblock = key

    block = (xblock, yblock, zblock)

    results = predValues[key]



    threadsPerBlock = xblock*yblock*zblock
    concurrentBlocks = min(32, 1024 // threadsPerBlock) * 80
    vMemComplete = results["memLoadISL"] * concurrentBlocks * threadsPerBlock
    sizeL2 = 6 * 1024 * 1024

    vMemStore = results["memStore"] * concurrentBlocks * threadsPerBlock
    vL2Store = results["L2Store"] * concurrentBlocks * threadsPerBlock
    vL2Load = results["L2Load"] * concurrentBlocks * threadsPerBlock

    vMem = vMemComplete



    if vMemStore > 0:
        vStoreEvicted = (vL2Store - vMemStore) * max(0, (vMemStore - sizeL2 * (vMemStore / (vMemStore + vMemComplete)) * min( 1,  0.5 * (block[0] * concurrentBlocks) / 2000)  )) / vMemStore
    else:
        vStoreEvicted = 0

    results["memStoreExt"] = ( vMemStore + vStoreEvicted) / concurrentBlocks / threadsPerBlock

    
    vMemEvicted = 0
    if vMemStore > 0:
        vMemEvicted += vStoreEvicted #(vL2Store - vMemStore) * max(0, (vMemStore - sizeL2 * (vMemStore / (vMemStore + vMemComplete))  * min( 1,  (block[0] * concurrentBlocks) / 2000)  )) / vMemStore           
        #vMemEvicted += (vL2Load - vMemComplete)
    if vMemComplete > 0:
        vMemEvicted += ((vL2Load - vMemComplete)
                       * max(0, (vMemComplete  - sizeL2 * (1 - 1 * vMemStore / vMemComplete)  * min( 1,  (block[0] * concurrentBlocks) / 2000) )/ vMemComplete))


    results["memLoadISLext"] = (vMem + vMemEvicted) / concurrentBlocks / threadsPerBlock
    #print(results["memLoadISLext"])

    results["memTotal"] = results["memLoadISLext"] + results["memStoreExt"]
    #print(vL2Load / 80 / 1024)
    #print( ((measValues[key]["memLoad"] * threadsPerBlock*concurrentBlocks) - vMemComplete) / (vL2Load - vMemComplete) )

In [ ]:
categories = ["L1", "L2", "RAM"]

   
keys = [k for k in measValues]

print(len(measValues))

volumeScatterPlot([(k[1:], measValues[k].lups, predValues[k].perfV1, categories[predValues[k].limV1]) for k in keys], "Multi Phase LBM Roofline range " + str(r) + " V1")
volumeScatterPlot([(k[1:], measValues[k].lups, predValues[k].perfV2, categories[predValues[k].limV2], predValues[k].perfV1) for k in keys], "Multi Phase LBM Roofline range " + str(r) + " V2")
volumeScatterPlot([(k[1:], measValues[k].lups, predValues[k].perfV3, categories[predValues[k].limV3], predValues[k].perfV2) for k in keys], "Multi Phase LBM Roofline range " + str(r) + " V3")
volumeScatterPlot([(k[1:], measValues[k].lups, predValues[k].perfV4, categories[predValues[k].limV4], predValues[k].perfV3) for k in keys], "Multi Phase LBM Roofline range " + str(r) + " V4")
volumeScatterPlot([(k[1:], measValues[k].lups, predValues[k].perfPheno, categories[predValues[k].limPheno], predValues[k].perfV4) for k in keys], "Multi Phase LBM Roofline range " + str(r) + " Pheno")



In [ ]:
volumeScatterPlot([(k[1:4], measValues[k].lups, min( predValues[k].perfPheno, (1000 + 400 / max(1, predValues[k].TLBpages / 1024) ) / (measValues[k].memLoad + measValues[k].memStore)),  k[3], predValues[k].perfPheno) for k in measValues.keys()  ], "testplot")
volumeScatterPlot([(k[1:4], measValues[k].lups, min( predValues[k].perfPheno, (2000 + 2500 / max(1, predValues[k].TLBpages / 64) ) / (measValues[k].L2Load_tex + measValues[k].L2Store)),  k[3], predValues[k].perfPheno) for k in measValues.keys()  ], "testplot")
volumeScatterPlot([(k[1:4], measValues[k].lups, min( predValues[k].perfPheno, 4500 / (1 + 2*(1 - min(1, 32 / predValues[k].TLBpages)) * (measValues[k].L2Load_tex + measValues[k].L2Store))),  k[3], predValues[k].perfPheno) for k in measValues.keys() if predValues[k].limPheno > 0 ], "testplot")


In [ ]:
top = [(m, measValues[m].lups) for m in measValues]
top.sort(key = lambda x : x[1])
for t in top[-8:]:
    print("{: >15s}: {:.2f}".format(str(t[0][1:4]), t[1]  ))

print()

top = [(m, predValues[m].perfV4) for m in predValues]
for t in top[-8:]:
    print("{: >15s}: {:.2f}".format(str(t[0][1:4]), t[1]  ))

